In [ ]:
import os
import numpy as np
import json
from bs4 import BeautifulSoup


def word_filter(words):      
   words = words.replace("\n", "")
   words = words.replace("\r", "")
   words = words.replace("&nbsp;", "")
   words = words.replace("	", "")
   words = words.replace(" ", "")
   words = words.strip()
   #s.replace('&nbsp;',' ')
   #article = re.sub(r'	', '', article)
   #article = re.sub(r' ', '', article)
   return words

def remove_subtitle(name):      
   name = name.replace(".html", "")
   name = name.replace(".pdf", "")
   name = name.replace(".docx", "")
   return name

# 轉換全部HTML成為JSON格式

# Handle files in the directory
html_dir = "./docs/input/html/"
json_dir = "./docs/output/0_json/"
paths = os.listdir('./docs/input/html/')


for files in paths:
    files_id = [] 
    titles_id = [] 
    intro_id = []
    author_id = []
    articles_id = []
    ref_id=[]

    try:
        with open(html_dir + files, encoding="utf-8") as f:            
            #print('\r\n'+ '檔案名稱:' + files)              
            files_id.append(files)

            soup = BeautifulSoup(f, "html.parser")                                    
            # 尋找以文件起點<div class="alohascontent" id="ctl00_divContent">   
            all_content = soup.find("div", class_ = "alohascontent")             
            # print (all_content)  # 列印網頁內容，包含html tag 
      
            # 文章題目                                
            try:
                title = all_content.find("p", class_ = "alohastitle").text   #找文章的題目
                #print('題目:' + files + word_filter(title))
                titles_id.append(word_filter(title)) 
            except:
                try:   
                    title = soup.find("div", class_ = "info-title").text   #找文章的題目
                    #print('The above code is a comment written in Chinese characters. It appears to be a placeholder for a question or topic.
                    題目:' + files + word_filter(title))
                    titles_id.append(word_filter(title)) 
                except:
                    print('錯誤-檔案找不到題目:' + files)
                    titles_id.append(np.nan)     
            
            # 文章引言                              
            try:
                introduction = all_content.find("em").text # 找文章引言<em>               
                #print('引言:' + word_filter(introduction))
                intro_id.append(word_filter(introduction))  
            except:
                try:
                    introduction = all_content.find("p", class_ = {"subtitle1","introduction"}).text # 找文章引言<em>
                    #print('引言:' + word_filter(introduction))
                    intro_id.append(word_filter(introduction))    
                except:
                    try:
                        introduction = all_content.find("p", class_ = "text-right author").findPrevious('p').text # 找文章引言<em>
                        #print('引言:' + word_filter(introduction))
                        intro_id.append(word_filter(introduction))
                    except:
                        try:
                            introduction = all_content.find("div", class_ = "clearfix").findNext('p').text # 找文章引言<em>
                            #print('引言:' + word_filter(introduction))
                            intro_id.append(word_filter(introduction))
                        except:
                            try:
                                introduction = soup.find("div", class_ = "content").findNext("div" , class_ = "wrap").findNext('p').text   #找文章的題目                                    
                                #print('引言:' + word_filter(introduction))
                                all_out.append(word_filter(introduction))
                            except:
                                #print('錯誤-文章找不到引言:' + files)
                                intro_id.append(np.nan)               
           
            # 作者
            try:
                author = all_content.find("p", class_ = {"text-right author","credit"}).text # 找文章引言<em>               
                #print('作者:' + word_filter(author))
                author_id.append(word_filter(author))  
            except:
                try:
                    author = all_content.find("div", class_ = {"subfont_12R"}).text # 找文章引言<em>               
                    #print('作者:' + word_filter(author))
                    author_id.append(word_filter(author))  
                except:
                    try:
                        author = all_content.find('span', attrs={"style": {"color:#8e44ad;font-size:0.933em;","color:#800080;font-size:0.933em;"}}).text # 找文章引言<em>               
                        #print('作者:"' + word_filter(author))
                        author_id.append(word_filter(author))  
                    except:
                        try:
                            author = soup.find("div", class_="content").findNext("div" , class_ = "wrap").findNext("div", class_="text-right").text   #找文章的題目                                    
                            #print('作者:' + word_filter(author))
                            author_id.append(word_filter(author)) 
                        except:
                            print('錯誤-文章找不到作者:' + files)
                            author_id.append(np.nan)

            # 內文/參考內文:
                  # 內文:

            try:
                for articles in all_content.find('p', attrs={"class": "text-right author"}).find_all_next("div",class_={"col-md-12","col-md-6"}):               
                    for articles1 in articles.find_all("p"):
                        #print('內文"1":' + word_filter(articles1.text))
                        articles_id.append(word_filter(articles1.text))
                for articles in all_content.find_all_next("div",class_={"caption","panel-body"}):                
                   #print('內文參考"1":' + word_filter(articles.text))  
                   ref_id.append(word_filter(articles.text))         
            except:
                try:
                    for articles in all_content.find('span', attrs={"style": {"color:#8e44ad;font-size:0.933em;"}}):
                        if articles.find_all_next("p", attrs={"align":"left"}):
                            for articles1 in articles.find_all_next("p", attrs={"align":"left"}):  
                                #print('內文"2":' + word_filter(articles1.text))
                                articles_id.append(word_filter(articles1.text))   
                        if articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs): 
                            for articles1 in articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs):  
                                #print('內文"2.1":' + word_filter(articles1.text))
                                articles_id.append(word_filter(articles1.text))
                                 
                        if articles.find_all_next("td", colspan={"2","3"}):
                            for articles1 in articles.find_all_next("td", colspan={"2","3"}):  
                                #print('內文參考"2.1":' + word_filter(articles1.text))
                                ref_id.append(word_filter(articles1.text))   
                        elif articles.find_all_next("p", style="text-align: left;"):
                            for articles1 in articles.find_all_next("p", style="text-align: left;"):  
                                #print('內文參考"2.2":' + word_filter(articles1.text)) 
                                ref_id.append(word_filter(articles1.text))  
                        elif articles.find_all_next("div", style="text-align: left;padding-left:520px;"):
                            for articles1 in articles.find_all_next("div", style="text-align: left;padding-left:520px;"):   
                                #print('內文參考"2.3":' + word_filter(articles1.text))
                                ref_id.append(word_filter(articles1.text))    
                        elif articles.find_all_next("span", style="font-size:0.933em;"):
                            for articles1 in articles.find_all_next("span", style="font-size:0.933em;"):   
                                #print('內文參考"2.4":' + word_filter(articles1.text))  
                                ref_id.append(word_filter(articles1.text))  

                except:
                    try:
                        for articles in all_content.find('span', attrs={"style": {"color:#800080;font-size:0.933em;"}}):
                            if articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs): 
                                for articles1 in articles.find_all_next(lambda tag: tag.name == 'p' and not tag.attrs):  
                                    #print('內文"3":' + word_filter(articles1.text))
                                    articles_id.append(word_filter(articles1.text))
                            #elif articles.find_all("p", attrs={"align":"left"}):                  
                            if articles.find_all_next("span", style="font-size:0.933em;"):
                                for articles1 in articles.find_all_next("span", style="font-size:0.933em;"):   
                                    #print('內文參考"3.1":' + word_filter(articles1.text))  
                                    ref_id.append(word_filter(articles1.text))                                     
                            elif articles.find_all_next("div", style="text-align: left;padding-left:520px;"):
                                for articles1 in articles.find_all_next("div", style="text-align: left;padding-left:520px;"):   
                                    #print('內文參考"3.2":' + word_filter(articles1.text)) 
                                    ref_id.append(word_filter(articles1.text))   
                    except:
                        try:
                            for articles in all_content.find('div', attrs={"class": "subfont_12R"}):
                                for articles1 in articles.find_all_next("p",attrs={"class":"subfont_13"}):          
                                    #print('內文"4":' + word_filter(articles1.text))
                                    articles_id.append(word_filter(articles1.text))
                            for articles in all_content.find('div', attrs={"class": "subfont_12R"}):
                                for articles1 in articles.find_all_next("div",attrs={"class":"subfont_12"}):     
                                    #print('內文參考"4":' + word_filter(articles1.text))
                                    ref_id.append(word_filter(articles1.text))                 
                        except:
                            try:
                                for articles in all_content.find("p", attrs={"class": "credit"}):                        
                                    for articles1 in articles.find_all_next("p"):                                          
                                        #print('內文"5":' + word_filter(articles1.text))
                                        articles_id.append(word_filter(articles1.text))
                                    for articles1 in articles.find_all_next("figcaption"):     
                                        #print('內文參考"5":' + word_filter(articles1.text))
                                        ref_id.append(word_filter(articles1.text))                   
                            except:
                                try:
                                    #print('內文"5":' + files)                                
                                    for articles in soup.find('div', attrs={"class": "content"}).find_all_next("p"):     
                                        #print('內文"6":' + word_filter(articles.text))
                                        articles_id.append(word_filter(articles.text))  
                                    for articles in soup.find('div', attrs={"class": "content"}).find_all_next("div",class_="caption"):                #         print(articles.text)
                                        #print('內文參考"6":' + word_filter(articles.text))
                                        ref_id.append(word_filter(articles.text))                              
                                except:
                                    #print('錯誤-文章找不到內文/參考:' + files)
                                    articles_id.append(np.nan) 
                                    ref_id.append(np.nan) 
  
        dict = {"filename": files_id, "titles": titles_id, "intro": intro_id, "author": author_id, "articles": list(filter(None, articles_id)), "reference": list(filter(None, ref_id))}
        with open(json_dir + remove_subtitle(files) + ".json", "w", encoding="utf-8") as f:
            f.write(json.dumps(dict, indent=4, ensure_ascii=False))

    except IOError as exc:
        raise



錯誤-檔案找不到題目:水源永續：多功能石門水庫的興建(204).html
錯誤-文章找不到作者:水源永續：多功能石門水庫的興建(204).html
